In [1]:
#Necessary imports for plotting, pandas, math operations, and multiple print statements.
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# read cleaned dataset
df = pd.read_csv("datasets/bettermovies.csv")

# get only necessary columns 
main_df = df[["originalTitle", "Scripts", "genres"]]
main_df = main_df[main_df["Scripts"].str.contains("<NA>") == False].reset_index().drop("index", axis=1)
main_df

,originalTitle,Scripts,genres
0,Kate & Leopold,time time proposed fourth dimension yet mortal...,"Comedy,Fantasy,Romance"
1,Barton Fink,kissing goodbye four walls six flights six fli...,"Comedy,Drama,Thriller"
2,Bill & Ted's Bogus Journey,suspenseful music playing time they've reached...,"Adventure,Comedy,Fantasy"
3,Billy Bathgate,irving makes cheap dago move weinberg man took...,"Crime,Drama,Thriller"
4,The Butcher's Wife,water across sea little island carolinas lived...,"Comedy,Fantasy,Romance"
...,...,...,...
2013,Hickok,keep goddamn head hear take gunner stay move g...,Western
2014,Lost in London,venom 5 million gtd poker tourney download mus...,"Comedy,Drama"
2015,A Bad Moms Christmas,blue christmas without blue alright christmas ...,Comedy
2016,SPF-18,real life movies little mixed ask sagittarius ...,"Drama,Romance,Sport"


In [3]:
# tf idf 
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

tfidf = TfidfVectorizer()
response = tfidf.fit_transform(main_df["Scripts"])
feature_names = tfidf.get_feature_names()
vectorized_df = pd.DataFrame(response.toarray()).set_axis(feature_names, axis=1)
vectorized_df

,00,000,0000,00000,0002,000500010000,001,0010,0012,0013,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# samp = vectorized_df.copy()
# samp.insert(0, "movie_titles", main_df["originalTitle"])
# samp = samp.set_index("movie_titles")
# samp.sample(n=5, axis=1).sample(n=5, axis=0)

In [5]:
# edit vectorized dataframe
vectorized_df.insert(0, "movie_genres", main_df["genres"].copy())
vectorized_df.insert(0, "movie_titles", main_df["originalTitle"].copy())

# divide into train and test
vectorized_train = vectorized_df.loc[:1800].copy()
vectorized_test = vectorized_df.loc[1801:].copy()

In [6]:
# splitting genres only for train
vectorized_train["movie_genres"] = vectorized_train["movie_genres"].squeeze().str.split(",")
vectorized_train = vectorized_train.explode("movie_genres")
vectorized_train

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
0,Kate & Leopold,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Kate & Leopold,Fantasy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Kate & Leopold,Romance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Barton Fink,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Barton Fink,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1799,Spectre,Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1799,Spectre,Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1800,Mission: Impossible - Rogue Nation,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1800,Mission: Impossible - Rogue Nation,Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# get 250 movies for each genre (Drama, Comedy, Action, Crime, Romance, Adventure, Thriller)
vectorized_train["movie_genres"].value_counts()
sorted_df = vectorized_train.sort_values(by=['movie_genres'], ascending=True)

not_remove = {'Action', 'Adventure', 'Comedy', 'Crime', 'Drama',  'Romance', 'Thriller'}
remove = {'Mystery', 'Fantasy', 'Sci-Fi', 'Biography', 'Animation', 'Family', 'Horror',  'Sport', 'Music', 'War', 'History', 'Musical', 'Western', 'Documentary'}

sorted_df = sorted_df[~sorted_df["movie_genres"].isin(remove)]

vectorized_train = vectorized_train.iloc[0:0]
for x in not_remove:
    temp = sorted_df.loc[sorted_df["movie_genres"] == x]
    vectorized_train = vectorized_train.append(temp.iloc[:300])

# get list of movies used for training (so we dont use them in testing)
train_movies = list(set(vectorized_train["movie_titles"].tolist()))

vectorized_train
vectorized_train["movie_genres"].value_counts()

Drama        1021
Comedy        743
Action        512
Crime         427
Romance       384
Adventure     360
Thriller      325
Mystery       181
Fantasy       145
Sci-Fi        131
Biography      95
Animation      94
Family         84
Horror         65
Sport          57
Music          44
History        14
War            14
Musical        12
Western         9
Name: movie_genres, dtype: int64

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
1201,Jack Reacher,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1389,From Paris with Love,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
842,Ladder 49,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
212,First Knight,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846,The Brothers Grimm,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,Revolutionary Road,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1290,Robin Hood,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130,True Romance,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,Sleepless in Seattle,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Action       300
Romance      300
Comedy       300
Thriller     300
Crime        300
Adventure    300
Drama        300
Name: movie_genres, dtype: int64

In [8]:
# convert "genres" from categorical to quantitative
d = {'Action': 0, 'Adventure': 1, 'Comedy': 3, 'Crime': 4, 'Drama': 5,  'Romance': 6, 'Thriller': 7}

vectorized_train["movie_genres"] = vectorized_train["movie_genres"].map(d)
vectorized_train

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
1201,Jack Reacher,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1389,From Paris with Love,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
842,Ladder 49,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
212,First Knight,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846,The Brothers Grimm,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,Revolutionary Road,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1290,Robin Hood,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130,True Romance,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,Sleepless in Seattle,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Convert genres in test to numbers and put them into lists

# remove movies that are in train set from test set
test_movies = list(set(vectorized_test["movie_titles"].tolist()))
for x in list(set(test_movies).intersection(train_movies)):
    vectorized_test = vectorized_test[vectorized_test["movie_titles"] != x]

# split movie_genres into one genre per line
vectorized_test_temp = vectorized_test.copy()
vectorized_test_temp["movie_genres"] = vectorized_test_temp["movie_genres"].squeeze().str.split(",")
vectorized_test_temp = vectorized_test_temp.explode("movie_genres")

# filter out movies from test that don't have the genres we're keeping
for x in remove:
    vectorized_test = vectorized_test[vectorized_test["movie_genres"] != x]
    vectorized_test_temp = vectorized_test_temp[vectorized_test_temp["movie_genres"] != x]

# numerize genres and put them into lists
vectorized_test_temp["movie_genres"] = vectorized_test_temp["movie_genres"].map(d)
vectorized_test_temp = vectorized_test_temp.groupby("movie_titles")["movie_genres"].apply(list).reset_index()

# lista = list(set(vectorized_test_temp["movie_titles"].tolist()))
# listb = list(set(vectorized_test["movie_titles"].tolist()))
# listc = list(set(lista) - set(listb))
# for x in listc:
#     vectorized_test = vectorized_test[vectorized_test["movie_titles"] != x]

# change string genres to numerized genres
vectorized_test["movie_genres"] = vectorized_test_temp["movie_genres"].values
vectorized_test

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
1864,5 Flights Up,[nan],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015,A Bad Moms Christmas,[nan],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1857,A Fighting Man,[nan],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1845,A Little Chaos,[nan],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1835,A Million Ways to Die in the West,[nan],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,Wonder Wheel,[nan],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1993,Wonderstruck,[nan],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1895,X-Men: Apocalypse,[nan],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1940,Yoga Hosers,[nan],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# knn 

from sklearn.neighbors import KNeighborsClassifier 
from numpy import dot
from numpy.linalg import norm
import math

X_train = vectorized_train.copy().drop(["movie_titles", "movie_genres"], axis=1)
y_train = vectorized_train["movie_genres"]
X_test = vectorized_test.copy().drop(["movie_titles", "movie_genres"], axis=1)
y_test = vectorized_test["movie_genres"]

# X_train
# X_test
# y_train
# y_test

def cos_sim(x, y):
    return dot(x, y) / (norm(x) * norm(y))

k = int(math.sqrt(X_train.shape[0]))

# knn_model = KNeighborsClassifier(n_neighbors=10, weights='distance', metric=cos_sim)
knn_model = KNeighborsClassifier(n_neighbors=15)
knn_model.fit(X=X_train, y=y_train)

In [ ]:
test_preds = knn_model.predict(X_test)
test_preds

# printing out neighbors and their distances
# distances, indices = knn_model.kneighbors(X_test)
# distances

In [ ]:
pd.set_option('display.max_rows', None)
# evaluate the performance of our model

# compute the error percentage of two arrays
# actual is an array of arrays of numbers
# predicted is an array of numbers
# return the number of correct predictions/total
def error_perc(actual, predicted):
    correct = 0;
    for x, y in zip(actual, predicted):
        for i in x:
            if y == i:
                correct = correct + 1
                # print(x, y)
                # print(str(correct) + ", " + str(len(actual)))
    return correct / len(actual)

ep = error_perc(y_test, test_preds)
ep

compare = vectorized_test[["movie_titles","movie_genres"]]
compare = compare.assign(predicted = pd.Series(test_preds).values)
compare
